<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/1.0.0-beta/TwelveLabs_Quickstart_Analyze.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Analyze videos

This guide shows how to utilize the TwelveLabs Python SDK to analyze videos and generate text based on their content.

## Key concepts

- **Index**: A container that organizes your video content
- **Asset**: Your uploaded file
- **Indexed asset**: A video that has been indexed and is ready for downstream tasks

## How it works

To analyze your videos, you must first upload and index them. The platform indexes videos asynchronously. After indexing completes, you can analyze your videos to generate text based on their content.

The upload method in this guide allows for files up to 4 GB when using publicly accessible URLs and 200 MB for local files. For details about the available upload methods and the corresponding limits, see the [Upload methods](https://docs.twelvelabs.io/docs/concepts/upload-methods) page.

**Customize text generation**

You can customize text generation in the following ways:
- Adjust the temperature to control the randomness of the output
- Set the maximum token limit in the response
Request structured JSON responses for programmatic processing
- Choose a response method based on your use case:
  - Streaming responses deliver text fragments in real-time as they are generated, enabling immediate processing and feedback. This method is the default behavior of the platform and is ideal for applications requiring incremental updates.
  - Non-streaming responses deliver the complete generated text in a single response, simplifying processing when the full result is needed.

# Prerequisites


- To use the platform, you need an API key:
  1. If you don't have an account, [sign up](https://playground.twelvelabs.io/) for a free account. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project.
  2. Go to the [API Keys](https://playground.twelvelabs.io/dashboard/api-keys) page.
  3. If you need to create a new key, select the **Create API Key** button. Enter a name and set the expiration period. The default is 12 months.
  4. Select the **Copy** icon next to your key to copy it to your clipboard.
- Your video files must meet the [format requirements](https://docs.twelvelabs.io/docs/concepts/models/pegasus#input-requirements).

# Procedure

## Install the TwelveLabs Python SDK

In [ ]:
%pip install twelvelabs

## Import the required packages

In [2]:
import time
from twelvelabs import TwelveLabs

## Configure your API key


In [ ]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get("TL_API_KEY")

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## Create an index

Indexes store and organize your video data, allowing you to group related videos. This guide shows how to create one, but you can also use an existing index. See the [Indexes](https://docs.twelvelabs.io/docs/concepts/indexes) page for more details on creating an index and specifying the model configuration.

In [ ]:
client = TwelveLabs(api_key=TL_API_KEY)

index = client.indexes.create(
    index_name="cb-pegasus-1",
    models=[{"model_name": "pegasus1.2", "model_options": ["visual", "audio"]}]
)
if not index.id:
    raise RuntimeError("Failed to create an index.")
print(f"Created index: id={index.id}")

## Upload a video

In [ ]:
asset = client.assets.create(
    method="url",
    url="<YOUR_VIDEO_URL>" # Example: https://github.com/twelvelabs-io/twelvelabs-developer-experience/raw/refs/heads/main/quickstarts/steve_jobs_introduces_iphone_in_2007.mp4
    # Or use method="direct" and file=open("<PATH_TO_VIDEO_FILE>", "rb") to upload a file from the local file system
)
print(f"Created asset: id={asset.id}")

## Index your video

In [ ]:
indexed_asset = client.indexes.indexed_assets.create(
    index_id=index.id,
    asset_id=asset.id,
    # enable_video_stream=True
)
print(f"Created indexed asset: id={indexed_asset.id}")

## Monitor the indexing process

In [ ]:
print("Waiting for indexing to complete.")
while True:
    indexed_asset = client.indexes.indexed_assets.retrieve(
        index_id=index.id,
        indexed_asset_id=indexed_asset.id
    )
    print(f"  Status={indexed_asset.status}")
    if indexed_asset.status == "ready":
        print("Indexing complete!")
        break
    elif indexed_asset.status == "failed":
        raise RuntimeError("Indexing failed")
    time.sleep(5)

## Analyze videos to generate text

### Streaming responses

In [ ]:
text_stream = client.analyze_stream(
    video_id=indexed_asset.id,
    prompt="<YOUR_PROMPT>", # Example: "Summarize this video"
    # temperature=0.2,
    # max_tokens=1024,
)

for text in text_stream:
    if text.event_type == "text_generation":
        print(text.text)

### Non-streaming responses

In [ ]:
text = client.analyze(
    video_id=indexed_asset.id,
    prompt="<YOUR_PROMPT>",
    # temperature=0.2,
    # max_tokens=1024,
)

print(f"{text.data}")

### Structured responses (streaming)

In [ ]:
from twelvelabs.types import ResponseFormat, StreamAnalyzeResponse_StreamEnd

text_stream = client.analyze_stream(
    video_id=indexed_asset.id,
    prompt="<YOUR_PROMPT>",
    # max_tokens=2048,
    response_format=ResponseFormat(
        type="json_schema",
        json_schema={
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "summary": {"type": "string"},
                "keywords": {"type": "array", "items": {"type": "string"}},
            },
        },
    ),
)
for chunk in text_stream:
    if chunk.event_type == "text_generation":
        print(chunk.text, end="", flush=True)
    elif isinstance(chunk, StreamAnalyzeResponse_StreamEnd):
        print(f"\nFinish reason: {chunk.finish_reason}")
        if chunk.metadata and chunk.metadata.usage:
            print(f"Usage: {chunk.metadata.usage}")

### Structured responses (non-streaming)

In [ ]:
import json
from twelvelabs.types import ResponseFormat

text = client.analyze(
    video_id=indexed_asset.id,
    prompt="<YOUR_PROMPT>",
    # max_tokens=2048,
    response_format=ResponseFormat(
        json_schema={
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "summary": {"type": "string"},
                "keywords": {"type": "array", "items": {"type": "string"}},
            },
        },
    ),
)
print(json.dumps(text.model_dump(), indent=2))
print (f"Finish reason: {text.finish_reason}")


# Next steps

For a comprehensive guide, see the [Analyze videos](https://docs.twelvelabs.io/docs/guides/analyze-videos) page.
